In [20]:
import argparse, os
import cv2
import numpy as np
import imageio
import time

In [2]:
def readPicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)#cv2.IMREAD_UNCHANGED is important that the output is (x,y,ChannelRGB)
    #print(img.shape)
    #alternative
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img)

In [3]:
def savePic(picture,fileName,extention,outPath):
    outPath = outPath+fileName+'.'+extention # combines the path with the name and extention of the file
    print(outPath)
    try:
        #imageio.imwrite(outPath,picture,format=extention)# old way
        cv2.imwrite(outPath,picture)#saves Pictures 
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(') #writes an error
        print('--------------------')

In [27]:
def YUVtoRGB(img):
    #pictureYUV = cv2.cvtColor(img, cv2.COLOR_YUV2RGB, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    im_ycbcr = img.astype(np.float32)
    im_ycbcr[:,:,0] = (im_ycbcr[:,:,0]*255.0-16)/(235-16) #to [0, 1]
    im_ycbcr[:,:,1:] = (im_ycbcr[:,:,1:]*255.0-16)/(240-16) #to [0, 1]
    im_ycrcb = im_ycbcr[:,:,(0,2,1)].astype(np.float32)
    im_rgb = cv2.cvtColor(im_ycrcb, cv2.COLOR_YCR_CB2BGR)#cv2.COLOR_YCR_CB2GBR)
    
    return im_rgb

In [5]:
def RGBtoYUV(img):
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from RGB to YU-V
    return pictureYUV

In [6]:
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min() # searches for the smalest number in the img array and saves it in imin
    imax = img.max() # searches for the biggest number in the img array and saves it in imax

    a = (target_type_max - target_type_min) / (imax - imin) # creates ratio of wanted to actual number value space
    b = target_type_max - a * imax # Creates the maximal possible value in b
    try:
        new_img = (a * img + b).astype(target_type) # recalculates the image with the calculated values and sets the new type
    except:
        print('error while converting the image')
    return new_img

In [7]:
def hdrorpng(extention,yuvPic):
    if (extention == 'hdr'): # when hdr than normalize in values between 0 and 1
        yuvPic = convert(yuvPic, 0, 1, np.float32) # send to convert in float 32 // Just devide by (2 ** 10) - 1?
    if (extention == 'png'): # when hdr than normalize in values between 0 and 255
        yuvPic = convert(yuvPic, 0, 255, np.uint8) # normalisation to unit8
    return yuvPic
    

In [8]:
#toDo Add parser with args
###### Imput section
print('Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png')
print('FILENAME MUST HAVE the format: 23-y_pred !')
path = input('Path to pictures who should be converted defaut: ./yuvPic/: ') or './yuvPic/'
inputextention = input('What fileextention do the to read pictures have? [default: png]') or 'png'
outputextention = input('Please type outputextention[default: hdr]: ') or 'hdr'
outputpath = input('Where to write the stiched pictures to? [default: ./hdrOut/]: ') or './hdrOut/'
namePic = input('What should be the name of the stiched pictures? [default: pred]') or 'predictedPic'
wantYUV = input('Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]') or 'no'
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention

Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png
FILENAME MUST HAVE the format: 23-y_pred !
Path to pictures who should be converted defaut: ./yuvPic/: /home/nico/mnt/cutRawData/jsiGanOutput/JSI-GAN_x2_exp1/
What fileextention do the to read pictures have? [default: png]
Please type outputextention[default: hdr]: 
Where to write the stiched pictures to? [default: ./hdrOut/]: 
What should be the name of the stiched pictures? [default: pred]assasas
Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]


In [29]:
#Working Version 10022021
start_time = time.time() #start Timer
#TO DO Parser
#desc ='yuv to RGB'
#parser = argparse.ArgumentParser(description=desc)
#parser.add_argument('--yuv', type=str, default='./', help='path to Folder of yuv images')
#print(parser.parse_args())
#what, b = parser.parse_known_args()

#if what.yuv == './' :
 #   print('yes')
start_time = time.time()
i = 0
print(aOp)
while (i <= aOp-1 ): # read y
    print(i)
    try:
        if ((str(os.listdir(path)[i]).split('-')[1]).split('_')[0]) == 'y': # only searching for y picitures
            name = os.listdir(path)[i] #finding the Name
            print(name +' should be the Y')
            picpath = path + name #combining Name and path
            picy = readPicture(picpath) #reads a pic y to find the x,y axes should be the same for all pictures
            #print(picy)
            yuvPic = np.zeros((int(picy.shape[0]),int(picy.shape[1]),3)) # generates the x and y achses and channels of picture
            yuvPic[:,:,0] = picy # packs the Y in pos 1
            num = (str(os.listdir(path)[i]).split('-')[0])#.split('_')[0]
            newPic = path + num + '-u_'+name.split('_')[1]
            picu = readPicture(newPic)#reads a picture with Ending U
            yuvPic[:,:,1] = picu # packs the u in pos 0
            newPic = path + num + '-v_'+name.split('_')[1] #Generates the Name for v
            picv = readPicture(newPic)#reads the new picture with ending V
            yuvPic[:,:,2] = picv # packs the u in pos 2
            #Right for the Testdata ist: 0y2u1v <<<<<<<<<<<<<

            if(wantYUV != 'y'):
                #yuvPic = hdrorpng(outputextention,yuvPic) # normalize after conversion
                yuvPic = yuvPic/((2**10)-1)
                rgbPic = YUVtoRGB(yuvPic) # to YUV to RGB conversion Matrix needs to have the Format Y[0] U[1] V[2]
                #bgrPic = bgrPic/((2**10)-1)
                #rgbPic = YUVtoBGR(yuvPic) # to YUV to RGB conversion Matrix needs to have the Format Y[0] U[1] V[2]
                savePic(rgbPic,(str(i)+namePic),outputextention,outputpath)#saves final RGB pic
                
                savePic(bgrPic,(str(i)+namePic+'BGR'),outputextention,outputpath)#saves final RGB pic
            if(wantYUV == 'y'):
                #yuvPic = hdrorpng(outputextention,yuvPic) # normalize after conversion
                yuvPic = YUVtoRGB(yuvPic)
                savePic(yuvPic,(str(i)+namePic),outputextention,outputpath)#saves final YUV pic
    except:
        print('')
    i = i + 1

print("--- %s seconds ---" % (time.time() - start_time))
print(str((time.time() - start_time)/60))
print('------------------------- Done --------------------')

90
0
1
2
08-y_pred.png should be the Y
./hdrOut/2assasas.hdr

3
4
06-y_pred.png should be the Y
./hdrOut/4assasas.hdr

5
6
21-y_pred.png should be the Y
./hdrOut/6assasas.hdr

7
8
29-y_pred.png should be the Y
./hdrOut/8assasas.hdr

9
10
14-y_pred.png should be the Y
./hdrOut/10assasas.hdr

11
12
13
14
15
16
17
18
24-y_pred.png should be the Y
./hdrOut/18assasas.hdr

19
20
18-y_pred.png should be the Y
./hdrOut/20assasas.hdr

21
22
23
12-y_pred.png should be the Y
./hdrOut/23assasas.hdr

24
25
19-y_pred.png should be the Y
./hdrOut/25assasas.hdr

26
27
23-y_pred.png should be the Y
./hdrOut/27assasas.hdr

28
29
16-y_pred.png should be the Y
./hdrOut/29assasas.hdr

30
31
32
01-y_pred.png should be the Y
./hdrOut/32assasas.hdr

33
34
35
36
09-y_pred.png should be the Y
./hdrOut/36assasas.hdr

37
38
03-y_pred.png should be the Y
./hdrOut/38assasas.hdr

39
30-y_pred.png should be the Y
./hdrOut/39assasas.hdr

40
41
42
43
44
11-y_pred.png should be the Y
./hdrOut/44assasas.hdr

45
46
47
48


In [28]:
YUVtoRGB(yuvPic)

array([[[-0.37116456,  0.2890378 ,  0.6126096 ],
        [-0.3248817 ,  0.34135875,  0.9111116 ],
        [-0.32480624,  0.28649974,  1.0455408 ],
        ...,
        [-0.24928568,  0.09636865,  0.38866508],
        [-0.28654456,  0.14526924,  0.28773755],
        [ 0.02510339,  0.26584744,  0.4994839 ]],

       [[-0.2933061 ,  0.23562722,  0.61170304],
        [-0.16991943,  0.17848359,  0.75312257],
        [-0.09168208,  0.07128412,  0.8920442 ],
        ...,
        [-0.38557157,  0.09784728,  0.28537044],
        [-0.28464854,  0.1588051 ,  0.3327823 ],
        [-0.03416187,  0.28054547,  0.4665364 ]],

       [[-0.34528613,  0.14813113,  0.76155347],
        [-0.29216322, -0.01623017,  0.8128936 ],
        [ 0.08482295, -0.02196774,  1.1298689 ],
        ...,
        [-0.4059083 ,  0.04232828,  0.38698304],
        [-0.35271293,  0.08764607,  0.25099266],
        [-0.24238096,  0.13891625,  0.3519404 ]],

       ...,

       [[-0.11885113,  0.14158061,  0.92423636],
        [ 0

In [11]:
yuvPic

array([[[0.32942326, 0.16422287, 0.69110459],
        [0.43695015, 0.12512219, 0.79960899],
        [0.44379277, 0.12121212, 0.87878788],
        ...,
        [0.18670577, 0.30694037, 0.65493646],
        [0.18181818, 0.2913001 , 0.59530792],
        [0.32746823, 0.36168133, 0.62170088]],

       [[0.30987292, 0.21407625, 0.70478983],
        [0.32942326, 0.26392962, 0.77908113],
        [0.31867058, 0.30889541, 0.87390029],
        ...,
        [0.14760508, 0.26197458, 0.61876833],
        [0.20039101, 0.28152493, 0.60997067],
        [0.32062561, 0.33626588, 0.60606061]],

       [[0.29912023, 0.1945259 , 0.80645161],
        [0.23460411, 0.25806452, 0.8856305 ],
        [0.34995112, 0.37829912, 1.        ],
        ...,
        [0.14369501, 0.25415445, 0.68523949],
        [0.13685239, 0.28445748, 0.60508309],
        [0.19941349, 0.3030303 , 0.6226784 ]],

       ...,

       [[0.3597263 , 0.27174976, 0.86412512],
        [0.38025415, 0.42033236, 0.83382209],
        [0.29814272, 0